In [9]:
import pandas as pd
from IPython.display import display

# Support Ticket Categorization

## Data Overview

In [11]:
df = pd.read_csv("Support_ticket_text_data_mid_term.csv")
print(f"Shape of dataframe: {df.shape}")
display(df.head())
counter = 0
for i in df["support_ticket_text"].isna():
    if i:
        counter += 1

if counter:
    print("Missing values are present")
    print(f"Number of missing values is {counter}")
else:
    print("No missing values present")

Shape of dataframe: (21, 2)


,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


No missing values present


## Model Building

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format